In [56]:
!pip install -U -q PyDrive
!pip install --upgrade --quiet pygsheets
from google.colab import auth
from oauth2client.client import GoogleCredentials
import google.auth
auth.authenticate_user()
credentials, _ = google.auth.default()
import pygsheets
pyc = pygsheets.client.Client(credentials)

import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

url = "https://en.wikipedia.org/wiki/Crime_in_New_York_City#Murders_by_year"

#Get Wikipedia page on murders per year in NYC
website_url = (requests
               .get(url)
               .text)

soup = BeautifulSoup(website_url, 'lxml')
tables = soup.find_all('table', {'class' : 'wikitable'})

tables_appended = []

#append the three wikipedia tables and find <td> in tags
for table in tables:
    table_list = []
    items = table.findAll('td')
    for item in items:
        clean_item = str(item).replace("/n", "")
        table_list.append(clean_item)
    tables_appended.extend(table_list)

years = []
murders = []

for item in tables_appended:
    elements = re.findall('>([^<>]*)<', item)
    #add element to murder list if there is a new-line character in the list or in the first element
    if "\n" in elements or "\n" in elements[0]:
        murders.append(elements[0].replace("\n", ""))
    else:
        years.append(elements[0])

df = pd.DataFrame({"year"       : years,
                   "#_murders"  : murders})

#Get demograhpic data
session = pyc.open_by_url("https://docs.google.com/spreadsheets/d/1TdcPsWHN5bOZsffAoaR4k9q5R745jn8mHNXS8zAEVE0/edit#gid=1848758818")
tab = session.worksheet_by_title("Sheet1")
demograhpic_data = tab.get_as_df(start = "D1", numerize = False)

#Merge demographic data with murder data
df = df.merge(demograhpic_data, how = "left", left_on = "year", 
              right_on = "Expanded_year").drop("Expanded_year", axis = 1)

df["#_murders"] = pd.to_numeric(df["#_murders"], errors = "coerce")
df["population"] = pd.to_numeric(df["population"], errors = "coerce")

#work out per 100K figure for murder
df["per_100k"] = (df["#_murders"].astype(float) / df["population"].astype(float)) * 100000

display(df)

,year,#_murders,population,per_100k
0,1928,404.0,5620048.0,7.188551
1,1929,425.0,5620048.0,7.562213
2,1930,494.0,6930446.0,7.127968
3,1931,588.0,6930446.0,8.484302
4,1932,579.0,6930446.0,8.354441
...,...,...,...,...
87,2016,335.0,8242624.0,4.064240
88,2017,292.0,8242624.0,3.542561
89,2018,289.0,8242624.0,3.506165
90,2019,311.0,8242624.0,3.773070


In [61]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.io import output_notebook

output_notebook()

df_vis = df.copy()
data_source = ColumnDataSource(data = df_vis)

p = figure(plot_width = 1500, plot_height = 800, toolbar_location = None)
p.line(x = "year", y = "#_murders", source = data_source, color = "#faae00", line_width = 2)
p.circle(x = "year", y = "#_murders", source = data_source, color = "#faae00", size = 3)

p.background_fill_color = '#232323'
p.border_fill_color = '#232323'
p.xgrid.grid_line_color = '#5a5a5a'
p.ygrid.grid_line_color = '#5a5a5a'
p.xaxis.axis_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "9pt"
p.xaxis.major_label_text_font_size = "9pt"
p.xaxis.major_label_text_color = "#eeeeee"
p.yaxis.major_label_text_color = "#eeeeee"
p.yaxis.axis_label_text_color = "#eeeeee"
p.xaxis.axis_label_text_color = "#eeeeee"
p.outline_line_color = None
p.xaxis.axis_line_color = "#eeeeee"
p.yaxis.axis_line_color = "#eeeeee"
p.xaxis.major_tick_line_color = "#eeeeee"
p.xaxis.minor_tick_line_color = "#eeeeee"
p.yaxis.major_tick_line_color = "#eeeeee"
p.yaxis.minor_tick_line_color = "#eeeeee"

p.yaxis.axis_label = "# Murders"
p.xaxis.axis_label = "Year"

show(p)

q = figure(plot_width = 1500, plot_height = 800, toolbar_location = None)
q.line(x = "year", y = "per_100k", source = data_source, color = "#faae00", line_width = 2)
q.circle(x = "year", y = "per_100k", source = data_source, color = "#faae00", size = 3)

q.background_fill_color = '#232323'
q.border_fill_color = '#232323'
q.xgrid.grid_line_color = '#5a5a5a'
q.ygrid.grid_line_color = '#5a5a5a'
q.xaxis.axis_label_text_font_size = "15pt"
q.yaxis.axis_label_text_font_size = "15pt"
q.yaxis.major_label_text_font_size = "9pt"
q.xaxis.major_label_text_font_size = "9pt"
q.xaxis.major_label_text_color = "#eeeeee"
q.yaxis.major_label_text_color = "#eeeeee"
q.yaxis.axis_label_text_color = "#eeeeee"
q.xaxis.axis_label_text_color = "#eeeeee"
q.outline_line_color = None
q.xaxis.axis_line_color = "#eeeeee"
q.yaxis.axis_line_color = "#eeeeee"
q.xaxis.major_tick_line_color = "#eeeeee"
q.xaxis.minor_tick_line_color = "#eeeeee"
q.yaxis.major_tick_line_color = "#eeeeee"
q.yaxis.minor_tick_line_color = "#eeeeee"

q.yaxis.axis_label = "# Murders per 100,000 population"
q.xaxis.axis_label = "Year"

show(q)